<div class = "row">
    <div class = "colums">
        <img src="..\trecslogo.png" align="left" alt="Drawing" width ="60"/>    
    </div>
    <div class = "colums">
        <img src="..\asrlogo.png" align="right" alt="Drawing" width ="175"/>
    </div>    
</div>

# SWAPTION

## Algemeen

### Import en instellingen

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, output_file, ColumnDataSource, output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter, FactorRange
from bokeh.io import show
output_notebook(hide_banner=True)

Links uitlijnen tabellen

In [5]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

Bij printen van een dataframe wordt slechts een beperkt aantal rijen getoond.

In [6]:
pd.set_option('display.max_rows', 300)

## Initialisatie parameters

In [7]:
# data
datumKwartaal = 20220331
datumActueel = 20220331 # in dit notebook veronderstel ik dat de actuele datum dezelfde is als de kwartaal datum
data = (datumKwartaal, datumActueel)

# naam van de basis curve
renteCurveBasis = 'FairValue'

## Importeer en bewerk data

### Rentecurves per kwartaal en actueel

In [10]:
df_curves = pd.read_excel(r"curvebestanden/curvesAssets.xlsx", decimal = '.')

df_curves['datum'] = datumKwartaal # in dit notebook dezelfde als actuele datum
datum_col = df_curves.pop('datum')
df_curves.insert(0, 'datum', datum_col)

pd.options.display.float_format = '{:,.8f}'.format
df_curves
#df_curves.to_excel('output.xlsx')

,datum,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,...,swap.cra.eur-stylized-1.zero.va.sw345.down345,swap.cra.eur-stylized-1.zero.va.sw345.up345,swap.cra.dnb,swap.cra.dnb.up,swap.cra.dnb.down,swap.cra.zero.va-ratio50.sw270,swap.cra.zero.va-ratio50.sw270.down270,swap.cra.zero.va-ratio50.sw270.up270,swap.cra.zero.va-ratio50.sw270_Currency_Up,swap.cra.zero.va-ratio50.sw270_Currency_Down
0,20220331,1,EUR,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,0.00919131,-0.00080869,...,-0.00080776,0.00919224,-0.00080869,0.00919131,-0.00080869,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869
1,20220331,2,EUR,0.00537754,0.00253036,0.01537754,0.00537754,0.00254137,0.01537754,0.00537754,...,0.00254794,0.01538412,0.00537754,0.01537754,0.00253036,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754
2,20220331,3,EUR,0.00806788,0.00411486,0.01806788,0.00806788,0.00412935,0.01806788,0.00806788,...,0.00413130,0.01806983,0.00806788,0.01806788,0.00411486,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788
3,20220331,4,EUR,0.00926595,0.00510382,0.01926595,0.00926595,0.00511846,0.01926595,0.00926595,...,0.00511910,0.01926659,0.00926595,0.01926595,0.00510382,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595
4,20220331,5,EUR,0.00990248,0.00577112,0.01990248,0.00990248,0.00578518,0.01990248,0.00990248,...,0.00578534,0.01990264,0.00990248,0.01990248,0.00577112,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248
5,20220331,6,EUR,0.01037414,0.00640208,0.02037414,0.01037414,0.00641534,0.02037414,0.01037414,...,0.00641525,0.02037405,0.01037414,0.02037414,0.00640208,0.01037414,0.00641534,0.02037414,0.01037414,0.01037414
6,20220331,7,EUR,0.01078707,0.00693993,0.02078707,0.01078707,0.00695264,0.02078707,0.01078707,...,0.00695239,0.02078682,0.01078707,0.02078707,0.00693993,0.01078707,0.00695264,0.02078707,0.01078707,0.01078707
7,20220331,8,EUR,0.01120210,0.00749941,0.02120210,0.01120210,0.00751155,0.02120210,0.01120210,...,0.00751119,0.02120174,0.01120210,0.02120210,0.00749941,0.01120210,0.00751155,0.02120210,0.01120210,0.01120210
8,20220331,9,EUR,0.01161711,0.00808305,0.02161711,0.01161711,0.00809461,0.02161711,0.01161711,...,0.00809416,0.02161665,0.01161711,0.02161711,0.00808305,0.01161711,0.00809461,0.02161711,0.01161711,0.01161711
9,20220331,10,EUR,0.01205979,0.00861073,0.02205979,0.01205979,0.00862207,0.02205979,0.01205979,...,0.00862154,0.02205925,0.01205979,0.02205979,0.00861073,0.01205979,0.00862207,0.02205979,0.01205979,0.01205979


### IMW bestand

In [15]:
df_imw = pd.read_csv(r"imwbestand/2022M03 Adjusted IMW Weekupdate 202205131221.csv",sep = ";",decimal = '.', encoding= 'unicode_escape', low_memory=False)
df_imw

,Versie CIC table,Versie rapport,Ecs Cons Ecap asr,ECS Cons SCR,RDS-STA account,RDS-STA accountname,Riskbearer,Eliminatie ASR,Eliminatie OTSO,CIC Id,...,sector nivo 5,Rating SII,Rating SII endorsement,Rating SII Ll,Portfolio-purpose,STS label,Tier,Derivaten type,Wal to call,Wal to final
0,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,AA,DEPOT,NaN,SEN,NaN,0.00000000,0.00000000
1,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,AA,DEPOT,NaN,SEN,NaN,0.00000000,0.00000000
2,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,AA,DEPOT,NaN,SEN,NaN,0.00000000,0.00000000
3,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,NL42,...,NaN,NaN,NaN,AA-,BEL,NaN,SEN,NaN,0.00000000,0.00000000
4,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL49,...,NaN,NaN,NaN,AA,BEL,NaN,NaN,NaN,0.00000000,0.00000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62021,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,BBB,BEL,NaN,NaN,NaN,0.00000000,0.00000000
62022,20211216,20201216,2528_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,BBB,BEL,NaN,NaN,NaN,0.00000000,0.00000000
62023,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,BBB,BEL,NaN,NaN,NaN,0.00000000,0.00000000
62024,20211216,20201216,2523_ECAP,NaN,S2_SK_C1014_A9A,Collective Investments Undertakings,ER,0,0,XL42,...,NaN,NaN,NaN,BBB,BEL,NaN,NaN,NaN,0.00000000,0.00000000


## Documentatie
In dit document staat de methode voor het berekenen van de swaptions beschreven. 

## Modellering

In [ ]:
def berekenTimingDelta(datumKwartaal, datum):    
    datumKwartaalDate = datetime.strptime(str(datumKwartaal), '%Y%m%d')
    datumDate = datetime.strptime(str(datum), '%Y%m%d')
    timingDelta = (datumDate - datumKwartaalDate).days/365
    return timingDelta

In [ ]:
def berekenForwardRente(t, renteCurve):
    if t == 1:
        forwardRente = renteCurve[0]
    else:
        forwardRente = (1+renteCurve[t-1])**t / (1+renteCurve[t-2])**(t-1) - 1
    return forwardRente

In [ ]:
def berekenForwardRentes(renteCurve):
    forwardRentes = []
    for t in range(0,99):
        forwardRente = berekenForwardRente(t+1, renteCurve)
        forwardRentes += [forwardRente]
    return forwardRentes

In [34]:
def berekenRentesTiming(valuta, datum, timing, renteCurve, curveType):
    rentes = df_curves[curveType].loc[(df_curves[curveType]['datum'] == datum) & (df_curves[curveType]['Currency'] == valuta), renteCurve].values
    jaren = range(0,100)
    rentesTiming = np.interp(timing, jaren, rentes)
    return rentesTiming

## Output

## Verificatie